## **Creating SQL Queries from Natural Language**

This tutorial will guide you through creating an application that generates SQL queries from natural language instructions and evaluates the quality of the generated queries. Along the way, you'll learn how to use Orq's deployment feature to enhance SQL generation. By the end of this tutorial, you'll be ready to experiment with SQL generation in your own projects.

Before starting, ensure you have an Orq account. If not, sign up at Orq.ai. Let's dive in!

**Step 1: Setting Up the Environment**  
The following commands install the required libraries for working with the Orq platform, handling datasets, and managing the SQL generation workflow.

## Import packages

In [ ]:
!pip install orq-ai-sdk datasets huggingface_hub

### Initializing the OrqAI Client

This code initializes the OrqAI client with an API key, either from the `ORQ_API_KEY` environment variable or a hardcoded default, and sets the environment to production.

In [ ]:
import os
from orq_ai_sdk import Orq

client = Orq(
  api_key=os.environ.get("ORQ_API_KEY", "YOUR_ORQ_API"),
)

### **Hugging Face**

Before proceeding, sign up for a free Hugging Face account if you don't already have one. You'll need an API key to access their datasets library. Retrieve your API key [here](https://huggingface.co/settings/tokens) after signing up or logging in.

**Step 3: Loading the Dataset**  
Use the Hugging Face datasets library to load a dataset containing table schemas and natural language instructions. Convert the dataset to a pandas DataFrame for easy manipulation.

In [ ]:
from huggingface_hub import login

# Use your Hugging Face API token
login(token="YOUR_HUGGING_FACE_API")

In [ ]:
from datasets import load_dataset

ds = load_dataset("Clinton/Text-to-sql-v1")

# Convert to a pandas DataFrame (selecting the "train" split as an example)
df = ds["train"].to_pandas()

# Select the top 50 rows
df = df.head(50)

# Display the DataFrame
print(df)

In [ ]:
df.columns

### **SQL Query Generation Use Case**
This deployment is designed to generate valid SQL queries based on specific table schemas and user-provided instructions. The model analyzes the instruction and the associated table schema to produce a precise and contextually appropriate SQL query.

SQL query generation is particularly useful when automating database interactions, building query assistants, or streamlining the process of accessing structured data through natural language inputs.

In [ ]:
df = df[["instruction", "input", "response"]]

**Step 4: Generating SQL Queries**

This step involves invoking the Orq deployment to generate SQL queries for each row in the dataset. The instruction column provides the natural language task, while the input column contains the table schema. The results are stored in a new column named output.

In [ ]:
# Initialize the outputs list
outputs = []

# Iterate through each row in the DataFrame
for _, row in df.iterrows():
    # Extract the 'instruction' and 'input' columns for each row
    instruction = row["instruction"]
    table = row["input"]

    # Invoke the deployment for each row
    generation = client.deployments.invoke(
        key="text_to_SQL",  # Replace with your actual deployment key
        context={
            "environments": []
        },
        inputs={
            "table": table,
            "instruction": instruction
        },
        metadata={
            "custom-field-name": "custom-metadata-value"
        }
    )

    # Append the model's output to the outputs list
    outputs.append(generation.choices[0].message.content)

# Add the outputs as a new column in the DataFrame
df["output"] = outputs

### Performance Check

**Step 5: Saving and Evaluating Results**  
Save the updated DataFrame containing the SQL queries to a file and evaluate their quality. Use metrics or manual inspection to verify the accuracy and relevance of the generated queries.

In [ ]:
df

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

true_labels = df["response"]
predicted_labels = df["output"]

# Calculate performance metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='macro', zero_division=0)
recall = recall_score(true_labels, predicted_labels, average='macro', zero_division=0)
f1 = f1_score(true_labels, predicted_labels, average='macro', zero_division=0)

# Print the metrics
print("Performance Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

**Next Steps**  
Congratulations! You've successfully built and tested a SQL generation application using Orq. To further enhance your project:

- Experiment with different datasets or deployment keys.
- Refine the prompt to improve SQL generation quality.
- Integrate the solution into a larger application for automated data access.

For more details and advanced features, visit the Orq documentation.